In [ ]:
### ICD and necroptosis gene expression analysis

In [ ]:
library(ggplot2)
read.table("Table5_ICD_gene.txt")%>%rownames()->ICD_gene
read.table("Table6_necrosis.txt")%>%rownames()->necrosi_gene
colnames(TCGA_risk)%>%intersect(.,c(ICD_gene,"risk"))%>%TCGA_risk[,.]->data_gene#TCGA_risk;entire_risk
data_gene = gather(data_gene,gene,exp,-risk)
library(RColorBrewer)
library(tidyr)
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
g3 = ggplot(data_gene,aes(gene,exp,fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "Gene expression") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=90,vjust = 0.7, hjust = 1))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
  stat_compare_means(aes(group = risk,label = ..p.signif..),method = "wilcox.test")
g3
ggsave("ICD_gene.pdf",g3,width = 10,height =5 )
colnames(TCGA_risk)%>%intersect(.,c(necrosi_gene,"risk"))%>%TCGA_risk[,.]->data_gene#TCGA_risk;entire_risk
data_gene = gather(data_gene,gene,exp,-risk)
g3 = ggplot(data_gene,aes(gene,exp,fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "Gene expression") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=90,vjust = 0.7, hjust = 1))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
  stat_compare_means(aes(group = risk, label = ..p.signif..), method = "wilcox.test")
g3
ggsave("necroptosis_gene.pdf", g3, width = 10, height =5 )

### Supplemenet Fig.5

### estimate

In [ ]:
library(estimate)
library(tidyverse)
library(RColorBrewer)
library(ggpubr)
load("../estimate/TCGA/estimate.rdata")
load("../../Model_lasso_cox/模型构建/芯片建模_TCGA验证/OKModel_risk.rdata")
Expr = data.frame(entire_risk[,-c(17662:17670)]%>%t())
Expr = data.frame(TCGA_exp_pd[,-c(1:10)]%>%t())
write.table(Expr,file = "TCGA_exp.txt",row.names = T,quote = F,sep = "\t")
filterCommonGenes(input.f = "TCGA_exp.txt", 
                  output.f = "TCGA_exp.gct", 
                  id = "GeneSymbol")
estimateScore(input.ds = "TCGA_exp.gct",
              output.ds = "TCGA_score_T.gct", 
              platform = "affymetrix")#illumina;affymetrix
scores <- read.table("TCGA_score_T.gct",skip = 2,header = T)
rownames(scores) <- scores[,1]
scores <- t(scores[,3:ncol(scores)])
scores
TCGA_estimate = scores
save(entire_estimate,TCGA_estimate,file = "estimate.rdata")


### ssgsea

In [ ]:
library(GSVA)
library(tidyverse)
library(data.table)
library(tidyr)
library(pheatmap)
library(RColorBrewer)
library(ggpubr)
load("Chip2_pd_exp for lasso_cox.rdata")
load("OKModel_risk.rdata")
entire_ = entire[,-c(17662:17668)]%>%t()
TCGA_exp_ = TCGA_exp_pd[,-c(1:10)]%>%t()
read.csv("29.csv") %>%gather(set,gene)%>%unique()->x
x = x[x$gene != "",]
geneset29 = split(x$gene,x$set)
ssGSEA_matrix <- gsva(expr = as.matrix(TCGA_exp_), 
                      gset.idx.list = geneset29,
                      method = 'ssgsea',kcdf = 'Gaussian',abs.ranking = TRUE)
scale_gsva_matrix <- t(scale(t(ssGSEA_matrix)))
scale_gsva_matrix[scale_gsva_matrix< -2] <- -2
scale_gsva_matrix[scale_gsva_matrix>2] <- 2
